**Set up modules/data**

In [224]:
import numpy as np
import pandas as pd
import math
import statsmodels.api as sm

pd.options.mode.copy_on_write = True
import random
from matplotlib.pyplot import subplots
import matplotlib.pyplot as plt
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize)

In [225]:
from ISLP import confusion_table
from ISLP.models import contrast
from sklearn.discriminant_analysis import \
     (LinearDiscriminantAnalysis as LDA,
      QuadraticDiscriminantAnalysis as QDA)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import RocCurveDisplay
roc_curve_est = RocCurveDisplay.from_estimator 
roc_curve_pred = RocCurveDisplay.from_predictions 
import statsmodels.api as sm



Load data set

In [226]:
UCL_key = pd.read_csv('../Data/key_stats.csv')

UCL_key

,player_name,club,position,minutes_played,match_played,goals,assists,distance_covered
0,Courtois,Real Madrid,Goalkeeper,1230,13,0,0,64.2
1,Vinícius Júnior,Real Madrid,Forward,1199,13,4,6,133.0
2,Benzema,Real Madrid,Forward,1106,12,15,1,121.5
3,Modrić,Real Madrid,Midfielder,1077,13,0,4,124.5
4,Éder Militão,Real Madrid,Defender,1076,12,0,0,110.4
...,...,...,...,...,...,...,...,...
742,Gil Dias,Benfica,Midfielder,1,1,0,0,0.7
743,Rodrigo Ribeiro,Sporting CP,Forward,1,1,0,0,0.7
744,Cojocari,Sheriff,Defender,1,1,0,0,0.5
745,Maouassa,Club Brugge,Defender,1,1,0,0,0.2


In [227]:
import pandas as pd
from functools import reduce

# --- Load all the CSVs ---
key_stats    = pd.read_csv('../Data/key_stats.csv')      # base
disciplinary = pd.read_csv('../Data/disciplinary.csv')
attacking    = pd.read_csv('../Data/attacking.csv')
attempts     = pd.read_csv('../Data/attempts.csv')
defending    = pd.read_csv('../Data/defending.csv')
distribution = pd.read_csv('../Data/distributon.csv')
goals        = pd.read_csv('../Data/goals.csv')          # skip goalkeeping.csv

# --- Helper: add a prefix to ALL non-key columns so names are unique ---
def prefix_cols(df, prefix):
    return df.rename(columns={
        col: f'{prefix}_{col}' for col in df.columns if col != 'player_name'
    })

# Give each dataframe its own column namespace
key_stats_p    = prefix_cols(key_stats,    'ks')
disciplinary_p = prefix_cols(disciplinary, 'disc')
attacking_p    = prefix_cols(attacking,    'att')
attempts_p     = prefix_cols(attempts,     'attmpt')
defending_p    = prefix_cols(defending,    'def')
distribution_p = prefix_cols(distribution, 'dist')
goals_p        = prefix_cols(goals,        'goal')

# --- Put them into a list ---
datasets = [
    key_stats_p,
    disciplinary_p,
    attacking_p,
    attempts_p,
    defending_p,
    distribution_p,
    goals_p,
]

# --- Merge everything on player_name using outer join ---
mega_UCL = reduce(
    lambda left, right: pd.merge(left, right, on='player_name', how='outer'),
    datasets
)

# Optional: fill missing values
mega_UCL = mega_UCL.fillna(0)
mega_UCL

,player_name,ks_club,ks_position,ks_minutes_played,ks_match_played,ks_goals,ks_assists,ks_distance_covered,disc_serial,disc_club,...,goal_position,goal_goals,goal_right_foot,goal_left_foot,goal_headers,goal_others,goal_inside_area,goal_outside_areas,goal_penalties,goal_match_played
0,Aaronson,Salzburg,Midfielder,715.0,8.0,0.0,2.0,103.5,326.0,Salzburg,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Abubakari,Malmö,Forward,116.0,4.0,0.0,0.0,15.3,146.0,Malmö,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Acuña,Sevilla,Defender,379.0,5.0,0.0,0.0,43.2,31.0,Sevilla,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Adams,Leipzig,Midfielder,292.0,5.0,0.0,0.0,38.9,146.0,Leipzig,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Adamu,Salzburg,Forward,231.0,8.0,1.0,0.0,32.6,55.0,Salzburg,...,Forward,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998,Óscar Rodríguez,Sevilla,Midfielder,22.0,1.0,0.0,0.0,3.8,0.0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
999,Čolak,Malmö,Forward,500.0,6.0,0.0,0.0,63,40.0,Malmö,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000,Šeško,Salzburg,Forward,234.0,6.0,0.0,0.0,32.1,263.0,Salzburg,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001,Šimić,Salzburg,Forward,4.0,1.0,0.0,0.0,1.3,0.0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Logistic Regression:**

In [228]:
def predict(X, model):
    # the built-in get_prediction tool returns an array, so we need to convert to a dataframe
    predictions_df = pd.DataFrame(model.get_prediction(X).predicted, columns=['y_hat'], index=X.index)
    return predictions_df['y_hat']

In [229]:
megaUCL = mega_UCL.drop_duplicates(subset=['player_name', 'ks_club']) #gets rid of dupelicate rows by looking at name and club

megaUCL


,player_name,ks_club,ks_position,ks_minutes_played,ks_match_played,ks_goals,ks_assists,ks_distance_covered,disc_serial,disc_club,...,goal_position,goal_goals,goal_right_foot,goal_left_foot,goal_headers,goal_others,goal_inside_area,goal_outside_areas,goal_penalties,goal_match_played
0,Aaronson,Salzburg,Midfielder,715.0,8.0,0.0,2.0,103.5,326.0,Salzburg,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Abubakari,Malmö,Forward,116.0,4.0,0.0,0.0,15.3,146.0,Malmö,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Acuña,Sevilla,Defender,379.0,5.0,0.0,0.0,43.2,31.0,Sevilla,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Adams,Leipzig,Midfielder,292.0,5.0,0.0,0.0,38.9,146.0,Leipzig,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Adamu,Salzburg,Forward,231.0,8.0,1.0,0.0,32.6,55.0,Salzburg,...,Forward,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998,Óscar Rodríguez,Sevilla,Midfielder,22.0,1.0,0.0,0.0,3.8,0.0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
999,Čolak,Malmö,Forward,500.0,6.0,0.0,0.0,63,40.0,Malmö,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000,Šeško,Salzburg,Forward,234.0,6.0,0.0,0.0,32.1,263.0,Salzburg,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001,Šimić,Salzburg,Forward,4.0,1.0,0.0,0.0,1.3,0.0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [230]:
megaUCL[megaUCL['player_name'].duplicated()]['player_name'].unique() #shows players with same last name

array(['Camara', 'Correa', 'Danilo', 'Diallo', 'Fernando', 'Henderson',
       'Herrera', 'João Mário', 'Karavaev', 'Martínez', 'Mendy', 'Onana',
       'Peña', 'Sarr', 'Steffen'], dtype=object)

In [ ]:
Dupes = mega_UCL[mega_UCL['player_name'].duplicated()]['player_name'].unique()  #shows rows of players with same last name

for name in Dupes:
    print("\n=== Rows for:", name, "===")
    display(megaUCL[megaUCL['player_name'] == name])

Find stastically significant features in data set

In [232]:
print(UCL_key.columns)
print(UCL_key['position'].unique().tolist())

Index(['player_name', 'club', 'position', 'minutes_played', 'match_played',
       'goals', 'assists', 'distance_covered'],
      dtype='object')
['Goalkeeper', 'Forward', 'Midfielder', 'Defender']


In [233]:
UCL_key['position'] = UCL_key['position'].map({'Defender': 0, 'Forward': 1, 'Goalkeeper': 0, 'Midfielder': 1})

X_UCL_key = UCL_key.drop(columns=['player_name', 'position', 'club'])
X_UCL_key = X_UCL_key.apply(pd.to_numeric, errors='coerce').fillna(0)
Y_UCL_key = UCL_key['position'].astype(int)

In [ ]:
glm = sm.GLM(Y_UCL_key,
             X_UCL_key,
             family=sm.families.Binomial())
results = glm.fit()
summarize(results)

#Assists not significant

,coef,std err,z,P>|z|
minutes_played,-0.0374,0.004,-9.489,0.000
match_played,0.2491,0.068,3.682,0.000
goals,1.4002,0.209,6.690,0.000
assists,0.1614,0.142,1.136,0.256
distance_covered,0.2792,0.034,8.133,0.000


In [235]:
probs_UCL=predict(X_UCL_key,results)

In [245]:
predictions_train = np.array([True]*len(Y_UCL_key))
predictions_train[probs_UCL<0.6] = False

In [246]:
confusion_table(predictions_train, Y_UCL_key)

Truth,0,1
Predicted,,
0,228,108
1,76,335


In [ ]:
#Now try with other variables from mega UCL

Run model to see what gives test data

Run k-fold to test data since less than 1000 observations

**K-means Clustering**

Normalize data

Run k-means, we can discuss what best one is or switch to hierarchial

In [238]:
#I was thinking 4 for 4 positions or we can research how many football archetypes there are

**Random Forest**

Run the random forest

In [ ]:
#See if random forest gives better results (misclassification rate)

Compare to logistic regression